<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class7-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 7.1**

**Announcements**
* If you haven't already, please watch Video No. 9: *The z-Transform*.
* Next PHL MTL event, tomorrow!
  * Tue, Nov. 1 at 5pm
  * Music Industry Recording STUDIO ONE
  * [Register here (by end of day)](https://bit.ly/phl-mtl)


# Optional stuff

## Interactive Matplotlib

Install `ipympl` for  Matplotlib


In [ ]:
#!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()
# %matplotlib widget

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
The usual modules...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal
import numpy.fft as fft
import pickle

rc('animation', html='jshtml')

path = '/content/drive/My Drive/eces435-work/class7.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'ou812'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helper functions

`myPlot()`

A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`myPlotFFT()`

Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, y_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim < 0:
      plt.ylim(y_lim, 0)
    elif y_lim > 0:
      plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`mySpectrogram()`

A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.stft(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 4*np.abs(Sxx) + 1e-15
  S_dBFS = 20*np.log10(S_mag)
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  # return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

# Load audio sample

* Happy Halloween!

In [ ]:
x, fs44 = sf.read(path + 'MJ.wav')
ipd.Audio(x, rate=fs44)

# Midterm: A general approach to STFT compression

In [ ]:
f1, t1, X = signal.stft(x, fs44, nperseg=1024, noverlap=512, nfft=1024)

S_mag = np.abs(X) + 1e-15
S_dB = 20*np.log10(S_mag)

plt.pcolormesh(t1, f1, S_dB)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

## Naive strategy (from class)

In [ ]:
remove_idx = np.where(S_dB < -45)
X_c = np.copy(X)
X_c[remove_idx] = 0

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(X_c)+1e-15))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

## Find the largest magnitudes per frame

In [ ]:
X_sort = np.sort(np.abs(X), axis=0)
X_sort[-10:,100]

In [ ]:
f2 = 513 - np.arange(513)
plt.pcolormesh(t1, f2, 20*np.log10(X_sort+1e-15))

In [ ]:
x_idx = np.argsort(np.abs(X), axis=0)
x_idx[-10:,100]

In [ ]:
def encode(x):
  f1, t1, X = signal.stft(x, 44100, nperseg=1024, noverlap=512, nfft=1024)
  n_freqs, n_frames = np.shape(X)
  N_c = 60

  X_mag = np.abs(X)
  x_sortIdx = np.argsort(X_mag, axis=0)
  
  fft_idx = np.int16(x_sortIdx[-N_c:,:])
  fft_real = np.zeros([N_c, n_frames], dtype='float16')
  fft_imag = np.zeros([N_c, n_frames], dtype='float16')

  for n in range(n_frames):
    X_sort = X[fft_idx[:,n], n]
 
    fft_real[:,n] = np.real(X_sort)
    fft_imag[:,n] = np.imag(X_sort)

  y_compressed = [fft_idx, fft_real, fft_imag]
  return y_compressed

In [ ]:
Y_c = encode(x)

In [ ]:
stream = pickle.dumps(Y_c)
c_bytes = len(stream)
orig_bytes = len(x) * 2
print('Compressed bytes: ', c_bytes )
print('Original bytes: ', orig_bytes )
print('Compression ratio: ', c_bytes / orig_bytes )

In [ ]:
def decode(y_compressed):
  fft_idx = y_compressed[0]
  fft_real = y_compressed[1]
  fft_imag = y_compressed[2]

  n_freqs, n_frames = np.shape(fft_idx)
  X_c = 1j * np.zeros([513, n_frames])

  for n in range(n_frames):
    X_c[fft_idx[:,n],n] = fft_real[:,n] + 1j*fft_imag[:,n]

  t_c, x_c = signal.istft(X_c, 44100, nperseg=1024, noverlap=512, nfft=1024)

  return x_c, X_c

In [ ]:
x_c, X_c = decode(Y_c)
ipd.Audio(x_c, rate=fs44)

In [ ]:
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(X_c)+1e-15))

# `firwin()` *Window* method of FIR filter design

In [ ]:
N = 128
f_c = 0.15
A_lp = signal.firwin(N, f_c)
plt.plot(A_lp)

In [ ]:
fig = myPlotFFT(A_lp, n_fft=2048)

## A sidenote on *side lobes*

The window is a multiplication in the time-domain
* $h_w[n] = h[n] \cdot w[n]$

That's a *convolution* in the frequency domain:
* $H_w[k] = H[k] * W[k]$

  * $H[k]$: The ideal filter response (brick wall)
  * $W[k]$: The DFT of the window

In [ ]:
# Ideal filter
L = 128

H_id = np.zeros(L)
n_c = f_c * L  # Convert Hz to a DFT index
print(n_c)

H_id[:int(n_c)] = 1
H_id[-int(n_c):] = 1

f = fs44 * np.arange(L) / L
plt.plot(f, H_id)
plt.xlabel('Frequency (Hz)')

In [ ]:
# Window (boxcar or rect function)
fig = myPlotFFT(np.ones(128), n_fft=2048)

In [ ]:
# Hanning window
fig = myPlotFFT(np.hanning(128), n_fft=2048)

## The impulse response

In [ ]:
delta = np.zeros(256)
delta[0] = 1

h_lp = signal.lfilter(A_lp, 1, delta)
plt.plot(h_lp)

In [ ]:
t = np.arange(256) / 10000
sine100 = np.sin(2 * np.pi * 100 * t)
plt.plot(sine100)

In [ ]:
y100 = signal.lfilter(A_lp, 1, sine100)
n = np.arange(256)
plt.plot(n,sine100, n, y100)

In [ ]:
f0 = 250

mySine = np.sin(2 * np.pi * f0 * t)

y500 = signal.lfilter(A_lp, 1, mySine)
fig = plt.figure()
plt.plot(n, mySine, n, y500)

In [ ]:
filename = '-' + str(f0) + '.png'
fig.savefig(path + 'figures/' + username + filename)

## Phase: what is it?
* Shift in output
* Relationship to frequency

In [ ]:
H_lp = fft.fft(A_lp, n=2048)
H_phase = np.angle(H_lp)
plt.plot(H_phase)
# plt.plot(np.unwrap(H_phase))

# Why use FIR Filters?
Pros:
* Easy to design
* Stable
* Can be linear phase

Cons:
* Need longer filters for sharp transitions
* Sidelobes, never flat or smooth stopband responses

# Why use IIR filters?

Pros:
* Sharper responses with fewer coefficients, due to feedback
* Smooth(er) magnitude response: Can eliminate side lobes

Cons:
* Can be unstable (due to feedback)
* Harder to design

